In [2]:
# Installation et chargement des requirements
import warnings
warnings.filterwarnings("ignore")

%pip install -r requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Unable to determine R home: [Errno 2] No such file or directory: 'R'
      cffi mode is CFFI_MODE.ANY
      Looking for R home with: R RHOME
      Unable to determine R home: [Errno 2] No such file or directory: 'R'
      R home found: None
      Error: rpy2 in API mode cannot be built without R in the PATH or R_HOME defined. Correct this or force ABI mode-only by defining the environment variable RPY2_CFFI_MODE=ABI
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
ERROR: Failed to build 'rpy2-rinterface' when getting requirements to build wheel
Note: you may need to restart the kernel to use updated packages.


# 1. Téléchargement des données et création du dataframe

Les données du medical panel expenditure ont été téléchargées depuis l'url
https://meps.ahrq.gov/mepsweb/data_stats/download_data_files.jsp.
 Nous allons à présent créer un dataframe nous permettant de mettre en oeuvre
les algorithmes de Machine Learning pour la prévision des dépenses de santé.

In [ ]:
# Nous chargeons les bibliothèques nécessaires pour pouvoir nous placer dans
# un environnement R pour la création du dataframe.
%load_ext rpy2.ipython


In [ ]:
%%R

# Chargement des packages
library(haven)
library(dplyr)
library(tidyr)
library(readr)

In [ ]:
%%R

# Augmentation de la limite de temps
options(timeout = 600)

# Nous créons une fonction pour le téléchargement des 11 fichiers que nous
#nous utiliserons
#Les arguments de cette fonction sont:
# - urls : les urls de téléchargement
# - zipnames : les noms des fichiers zip à télécharger
# - objnames : une liste contenant les nouveaux noms attribués à ces fichiers
# - outdir : le nom du dossier où seront stockés les fichiers renommés

load_meps_zip <- function(urls, zipnames, objnames, outdir = "meps_data") {

  #création du dossier outdir
  dir.create(outdir, showWarnings = FALSE)

  for (i in seq_along(urls)) {
    url      <- urls[i]
    zipfile  <- file.path(outdir, zipnames[i])
    objname  <- objnames[i]

    #  Téléchargement du fichier zip
    download.file(url, destfile = zipfile, mode = "wb")

    #  Décompression et extraction du fichier .dta
    unzip_dir <- file.path(outdir, tools::file_path_sans_ext(zipnames[i]))
    dir.create(unzip_dir, showWarnings = FALSE)
    unzip(zipfile, exdir = unzip_dir)
    dta_file <- list.files(unzip_dir, pattern = "\\.dta$", full.names = TRUE)

    # Nous téléchargeons le fichier .dta et le renommons
    assign(objname, read_dta(dta_file), envir = .GlobalEnv)
  }
}

# Application de la fonction -------------------------------------------

urls <- c(
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h243/h243dta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h252/h252dta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h251/h251dta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h239a/h239adta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h239b/h239bdta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h239c/h239cdta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h239d/h239ddta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h239e/h239edta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h239f/h239fdta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h239g/h239gdta.zip",
  "https://meps.ahrq.gov/mepsweb/data_files/pufs/h239h/h239hdta.zip"
)

zipnames <- c("h243.zip", "h252.zip", "h251.zip", "h239a.dta", "h239b.dta",
              "h239c.dta", "h239d.dta", "h239e.dta", "h239f.dta", "h239g.dta",
              "h239h.dta")

objnames <- c("full_year_2022", "longitudinal", "full_year_2023",
              "prescribed_medicines_2022", "dental_2022","others_2022",
              "hospitals_2022", "emergency_room_2022", "outpatients_visits_2022",
              "Office_Based_Medical_Provider_2022", "home_health_2022")

load_meps_zip(urls, zipnames, objnames)




In [ ]:
%%R

# Nous agrégeons les dépenses pour chaque individu et pour chaque mois
# dans les fichiers détaillant les dépenses par poste de façon à obtenir
# - les dépenses mensuelles de chaque individu par poste
#(variables de type exp_poste_1, exp_poste2... pour le poste "poste")
# - une variable détaillant la dépense totale en 2022 pour ce poste pour chaque
# individu (variable exp_poste_total)

# Les cinq premiers fichiers sont traités de façon analogue : on fait une fonction
aggregate_by_season <- function(data, date_col, exp_col, prefix) {
  # Nom des fichiers et colonnes
  date_col_sym <- sym(date_col)
  exp_col_sym <- sym(exp_col)
  total_col_name <- paste0(prefix, "total")
  
  result <- data %>%
    #nous créons une variable de saison
    mutate(
      SAISON = case_when(
        !!date_col_sym %in% c(1, 2, 3) ~ "winter",
        !!date_col_sym %in% c(4, 5, 6) ~ "spring",
        !!date_col_sym %in% c(7, 8, 9) ~ "summer",
        !!date_col_sym %in% c(10, 11, 12) ~ "fall",
        TRUE ~ NA_character_
      )
    ) %>%
    #nous sommons les dépenses par saison et par individu
    group_by(DUPERSID, SAISON) %>%
    summarise(
      exp_saison = sum(!!exp_col_sym, na.rm = TRUE),
      .groups = "drop"
    ) %>%
    # Nous créons une colonne de dépense par saison
    pivot_wider(
      names_from = SAISON,
      values_from = exp_saison,
      names_prefix = prefix,
      values_fill = 0 # on remplace les NA (dépense nulle dans une saison) par 0
    ) %>%
    # Nous ajoutons la somme annuelle
    rowwise() %>%
    mutate(
      !!total_col_name := sum(c_across(starts_with(prefix)), na.rm = TRUE)
    ) %>%
    ungroup()
  
  return(result)
}


# Dépenses dentaires
agg_dental <- aggregate_by_season(
  data = dental_2022,
  date_col = "DVDATEMM",
  exp_col = "DVXP22X",
  prefix = "exp_dental_"
)

# Dépenses des visites externes
agg_outpatient <- aggregate_by_season(
  data = outpatients_visits_2022,
  date_col = "OPDATEMM",
  exp_col = "OPXP22X",
  prefix = "exp_outpatient_"
)

# Dépenses des cabinets médicaux
agg_office <- aggregate_by_season(
  data = Office_Based_Medical_Provider_2022,
  date_col = "OBDATEMM",
  exp_col = "OBXP22X",
  prefix = "exp_office_"
)

# Dépenses des urgences
agg_er <- aggregate_by_season(
  data = emergency_room_2022,
  date_col = "ERDATEMM",
  exp_col = "ERXP22X",
  prefix = "exp_er_"
)

# Dépenses de soins à domicile
agg_home <- aggregate_by_season(
  data = home_health_2022,
  date_col = "HHDATEMM",
  exp_col = "HHXP22X",
  prefix = "exp_home_"
)





In [ ]:
aggregate_uniform_seasonal <- function(data, exp_col, prefix) {
  # Nom des df et colonnes
  exp_col_sym <- sym(exp_col)
  total_col_name <- paste0(prefix, "total")
  
  seasons_list <- c("winter", "spring", "summer", "fall")
  
  seasonal_data <- data %>%
    # on calcule la dépense pour chaque saison
    mutate(exp_seasonal = !!exp_col_sym / 4) %>%
    # Création de 4 lignes par individu (une pour chaque saison)
    tidyr::crossing(SAISON = seasons_list) %>%
    group_by(DUPERSID, SAISON) %>%
    summarise(
      # Somme de la dépense saisonnière calculée
      exp_final = sum(exp_seasonal, na.rm = TRUE), 
      .groups = "drop"
    )
  
  # Calcul de la dépense par saison et la dépense totale pour chaque individu
  agg_data <- seasonal_data %>%
    pivot_wider(
      names_from = SAISON,
      values_from = exp_final,
      names_prefix = prefix,
      values_fill = 0
    ) %>%
    rowwise() %>%
    mutate(
      !!total_col_name := sum(c_across(starts_with(prefix)), na.rm = TRUE)
    ) %>%
    ungroup()
  
  return(agg_data)
}

# Traitement du fichier 'medicines'
agg_medicines <- aggregate_uniform_seasonal(
  data = prescribed_medicines_2022,
  exp_col = "RXXP22X",
  prefix = "exp_medicines_"
)

# Traitement du fichier 'others'
agg_others <- aggregate_uniform_seasonal(
  data = others_2022,
  exp_col = "OMXP22X",
  prefix = "exp_others_"
)

In [ ]:
%%R

#Le fichier hospitals nécessite un traitement particulier puisque les mois
#où la dépense est effectuée ne sont pas détaillés. A contrario, le fichier
# contient la date d'entrée et de sortie, nous répartirons donc uniformément la
# dépense entre les mois concernés par l'hospitalisation puis on regroupe par saison

#Répartition uniforme de la dépense entre le mois de début et le mois de fin de
#l'hospitalisation et création des variables exp_hospitals_month
# (dépenses d'hoispitalisation mensuelles)


# Fonction utilitaire pour mapper le mois à la saison
get_saison_from_month <- function(month) {
  case_when(
    month %in% c(1, 2, 3) ~ "winter",
    month %in% c(4, 5, 6) ~ "spring",
    month %in% c(7, 8, 9) ~ "summer",
    month %in% c(10, 11, 12) ~ "fall",
    TRUE ~ NA_character_
  )
}



hospitals_monthly <- hospitals_2022 %>%
  #On supprime les lignes pour lesquelles la date de fin d'hospitalisation n'est pas connue
  filter(IPENDMM != -8) %>%
  # Calcul du nombre de mois d'hospitalisation selon l'année de début
  mutate(
    n_months = case_when(
      IPBEGYR == 2022 ~ IPENDMM - IPBEGMM + 1,
      IPBEGYR == 2021 ~ (12 - IPBEGMM + 1) + IPENDMM,
      TRUE ~ NA_real_
    ),
    #Calcul de la dépense mensuelle
    exp_hospitals_month = IPXP22X / n_months
  ) %>%
  rowwise() %>%
  #Liste des mois concernés par l'hospitalisation
  mutate(
    month = case_when(
      IPBEGYR == 2022 ~ list(seq(IPBEGMM, IPENDMM)),
      IPBEGYR == 2021 ~ list(seq(1, IPENDMM)), 
      TRUE ~ list(NA_integer_)
    )
  ) %>%
  unnest(month) %>%
  ungroup() %>%
  # Création de la variable SAISON à partir du mois
  mutate(SAISON = get_saison_from_month(month))

# Agrégation des dépenses mensuelles en dépenses saisonnières (pour 2022)
hospitals_seasonal <- hospitals_monthly %>%
  group_by(DUPERSID, SAISON) %>%
  # On somme toutes les contributions mensuelles de chaque saison
  summarise(exp_hospitals = sum(exp_hospitals_month, na.rm = TRUE), .groups = "drop") %>%
  # On complète le dataframe pour s'assurer que toutes les 4 saisons sont présentes (avec 0 si aucune dépense)
  complete(DUPERSID, SAISON = c("winter", "spring", "summer", "fall"), fill = list(exp_hospitals = 0))

# Création de la dépense agrégée finale par saison 
agg_hospital <- hospitals_seasonal %>%
  pivot_wider(
    names_from = SAISON,
    values_from = exp_hospitals,
    names_prefix = "exp_hospitals_",
    values_fill = 0 
  ) %>%
  rowwise() %>%
  # Total de la dépense imputée à 2022
  mutate(
    exp_hospitals_2022_total = sum(
      c_across(starts_with("exp_hospitals_")),
      na.rm = TRUE
    )
  ) %>%
  ungroup()



In [ ]:
%%R

# Les variables précédemment créées détaillant les dépenses agrégées par poste
# pour chaque individu sont jointes aux fichiers full_year_2022 et longitudinal
# pour inclure certaines caractéristiques socio-démographiques et économiques à l'étude.

# DUPERSID présents dans hospitals_2022 mais supprimés dans agg_hospitals
# (du fait de l'absence de l'information sur la durée d'hospitalisation)
ids_removed <- hospitals_2022 %>%
  filter(IPENDMM == -8 ) %>%
  pull(DUPERSID)


# CRéation du dataframe avec l'ensemble des dépenses mensuelles par postes
# et les caractéristiques socio-démographiques et économiques
all_expenses <- full_year_2022 %>%
  filter(!DUPERSID %in% ids_removed)  %>%
  select(DUPERSID,  EDUCYR,  TTLP22X, FAMINC22, AGE22X, TOTEXP22  ) %>%
  left_join(full_year_2023 %>% select(DUPERSID,  TOTEXP23), by= "DUPERSID") %>%
  left_join(longitudinal %>% select(DUPERSID), by= "DUPERSID")%>%
  left_join(agg_dental, by="DUPERSID") %>%
  left_join(agg_hospital, by="DUPERSID") %>%
  left_join(agg_outpatient, by="DUPERSID") %>%
  left_join(agg_office, by="DUPERSID") %>%
  left_join(agg_er, by="DUPERSID") %>%
  left_join(agg_home, by="DUPERSID") %>%
  left_join(agg_others, by="DUPERSID") %>%
  left_join(agg_medicines, by="DUPERSID")




In [ ]:
%%R

# Les valeurs démographiques négatives indiquent que la modalité est inconnue
# pour l'individu concerné d'après la documentation 

# Etude sur le nombre d'années d'éducation
print(table(all_expenses$EDUCYR))
na_educ <- all_expenses %>% filter(EDUCYR <0)

# On regarde l'âge de ces individus
print(table(na_educ$AGE22X))
# Il semblerait que cela soit des jeunes : on compare les jeunes avec un EDUCYR à ceux sans :

young <- all_expenses %>% filter(0<= AGE22X & AGE22X < 9)
print(table(young$AGE22X))
young_educ <- young %>% filter(EDUCYR>=0)
print(table(young_educ$AGE22X))
# On a donc tous les moins de 4 ans qui ont une modalité négative : on va la mettre à 0

all_expenses_clean <- all_expenses %>% mutate(
    EDUCYR = ifelse( EDUCYR <0, 0, EDUCYR)
)


## Si on veut un joli graphe :
library(ggplot2)
plot_data <- all_expenses %>%
  mutate(
    EDUCYR_status = case_when(
      EDUCYR < 0 ~ "EDUCYR Négatif/Inconnu",
      EDUCYR == 0 ~ "EDUCYR = 0 (Aucune éducation)",
      EDUCYR > 0 ~ "EDUCYR Positif",
      TRUE ~ "NA"
    ),
    Age_Group = ifelse(AGE22X < 9, "Très Jeune (0-8 ans)", "Adultes et Enfants (9+ ans)")
  ) %>%
  # On filtre pour les groupes pertinents
  filter(EDUCYR_status != "NA")

density_data <- all_expenses %>%
  # Limiter l'analyse aux individus de moins de 18 ans
  filter(AGE22X < 18) %>% 
  mutate(
    EDUCYR_status = ifelse(EDUCYR < 0, "EDUCYR Négatif/Inconnu", "EDUCYR Non-Négatif (>= 0)")
  )

%%R
# Graphique de densité pour comparer les distributions d'âge
ggplot(density_data, aes(x = AGE22X, fill = EDUCYR_status)) +
  geom_density(alpha = 0.5, adjust = 1) + # adjust=1 pour lisser l'estimation de densité
  scale_fill_manual(values = c("EDUCYR Négatif/Inconnu" = "#E41A1C", 
                               "EDUCYR Non-Négatif (>= 0)" = "#377EB8")) +
  labs(
    title = "Comparaison de la Distribution d'Âge (Moins de 18 ans)",
    subtitle = "EDUCYR Négatif (Cas à imputer à 0) vs. EDUCYR Non-Négatif",
    x = "Âge en 2022 (AGE22X)",
    y = "Densité",
    fill = "Statut de EDUCYR"
  ) +
  theme_minimal()



In [ ]:
%%R

# On a quelques outliers avec income négatif et d'autres sans donnée d'âge : on les supprime
cat("\n Nombre de lignes total \n")
print(nrow(all_expenses_clean))
cat("\n Nombre de lignes avec revenu négatif \n")
print(nrow(all_expenses_clean %>% filter(TTLP22X <0 | FAMINC22 <0)))
cat("\n Nombre de lignes sans donnée d'âge \n")
print(nrow(all_expenses_clean %>% filter(AGE22X <0)))

all_expenses_clean <- all_expenses_clean %>% 
  filter(TTLP22X >=0 & FAMINC22 >=0 & AGE22X >=0)

In [ ]:
  %%R
# Nous créons d'autres variables utiles pour notre étude, à savoir:
# - Les dépenses des 6 derniers mois (dep_6_mois)
# - Les dépenses des 3 derniers mois (dep_3_mois)
# - Nombre de mois où la dépense est supérieure à la dépense mensuelle moyenne (nbre_au_dessus_moyenne)
# - Création d'une variable tendance obtenue en ajustant une droite (régression linéaire)
# sur les dernières dépenses mensuelles de la période d’observation, puis en extrayant la pente.
# - Création de la variable ep_aigue qui indique si la dépense mensuelle la plus élevée est
#supérieure au triple de la dépense mensuelle moyenne
# -Création de la variable dépense mensuelle maximale dep_max

#Nous créons une liste contenant les variables de dépenses mensuelles
dep_months <- c(
  "dep_janvier", "dep_fevrier", "dep_mars", "dep_avril",
  "dep_mai", "dep_juin", "dep_juillet", "dep_aout",
  "dep_sept", "dep_oct", "dep_nov", "dep_dec"
)

  # Nous créons la variable dépense sur les 3 derniers mois
  all_expenses_clean <- all_expenses_clean %>%
  mutate(
    dep_3_mois = rowSums(across(matches("_(10|11|12)$")), na.rm = TRUE)
  )

  # Nous créons la variable dépense sur les 6 derniers mois
  all_expenses_clean <- all_expenses_clean %>%
  mutate(
    dep_6_mois = rowSums(across(matches("_(7|8|9|10|11|12)$")), na.rm = TRUE)
  )

  # Nous créons la variable nbre_au_dessus_moyenne
  all_expenses_clean <- all_expenses_clean %>%
  rowwise() %>%
  mutate(
    # calcul du nombre de mois dont la dépense dépasse la moyenne
    nbre_au_dessus_moyenne =
      sum(c_across(dep_janvier:dep_dec) >  (TOTEXP22 / 12), na.rm = TRUE)
  ) %>%
  ungroup()

#Nous créons la variable tendance
all_expenses_clean <- all_expenses_clean %>%
  rowwise() %>%
  mutate(
    tendance = {
      #Selection des trois derniers mois
      mois <- 1:3
      dep_mensuelles <- c(dep_oct, dep_nov, dep_dec)
      # Calcul de la pente de la régression linéaire
      lm(dep_mensuelles ~ mois)$coefficients[2]
    }
  ) %>%
  ungroup()

# Nous créons les variables ep_aigue et dep_max
all_expenses_clean <- all_expenses_clean %>%
  rowwise() %>%
  mutate(
    # Dépense mensuelle maximale
    dep_max     = max(c_across(all_of(dep_months)), na.rm = TRUE),
    ep_aigue    = if_else(dep_max > 5 *TOTEXP22 / 12, 1, 0)
  ) %>%
  ungroup()



In [ ]:
%%R

# Nous créons les variables ep_aigue et dep_max
all_expenses_clean <- all_expenses_clean %>%
  rowwise() %>%
  mutate(
    # Dépense mensuelle maximale
    dep_max     = max(c_across(all_of(dep_months)), na.rm = TRUE),
    # Présence d'une dépense mensuelle aigue: supérieure à 5 fois la dépense moyenne
    ep_aigue    = if_else(dep_max > 5 * TOTEXP22 / 12, 1, 0)
  ) %>%
  ungroup()


In [ ]:
%%R

# Nous enregistrons le dataframe crée
write.csv(all_expenses_clean, "all_expenses_clean.csv", row.names = FALSE)

# 2. Familiarisation avec les données: quelques statistiques descriptives

In [ ]:
# Nous chargeons les packages nécessaires

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn


In [ ]:
# Nous lisons le fichier all_expenses_clean créé précédemment avec python
all_expenses_clean = pd.read_csv("all_expenses_clean.csv", sep=",")

# Nous vérifions quelles colonnes contiennent des valeurs manquantes
pd.set_option('display.max_rows', None)#  option pour voir toutes les entrées de la Series
all_expenses_clean.count()
#pd.reset_option('display.max_rows') #nous supprimons l'option

# Nous remarquons que les variables socio-démographiques et économiques ne contiennent pas de NA.
# Les seules variables contenant des NA sont les dépenses par poste et seront donc remplacées par 0.



Nous commencons par réaliser un histogramme détaillant la répartiton des dépenses de santé en 2022 par poste de dépense (au niveau de notre population).

Nous remarquons que les dépenses de pharmacie (exp_medicines_total), les consultations médicales (exp_home_total) et les dépenses hospitalières (exp_hospitals_2022_total, exp_outpatient_total) constituent les postes de dépenses les plus importants.

In [ ]:
# Liste des postes à agréger
depenses_postes = [
    "exp_dental_total",
    "exp_hospitals_2022_total",
    "exp_outpatient_total",
    "exp_er_total",
    "exp_medicines_total",
    "exp_office_total",
    "exp_home_total",
    "exp_others_total"
]

# Calcul les dépenses totales par poste (somme sur la population)
total_depenses_postes = all_expenses_clean[depenses_postes].sum()

# Création du graphique
plt.figure(figsize=(12, 6))
plt.bar(total_depenses_postes.index,total_depenses_postes.values)

# Titre et légendes
plt.title("Répartition des dépenses de santé par poste (niveau population)", fontsize=14)
plt.xlabel("Poste de dépense", fontsize=12)
plt.ylabel("Dépenses totales (USD)", fontsize=12)

# Rotation des labels pour plus de lisibilité
plt.xticks(rotation=45, ha='right')

# Affichage
plt.tight_layout()
plt.show()






Nous réalisons ensuite un histogramme détaillant la répartiton des dépenses de santé en 2022 par mois (au niveau de notre population).

Nous remarquons que les dépenses de santé sont relativement stables au cours de l'année.

In [ ]:
# Liste des postes à agréger
var_mensuelles = [
 "dep_janvier", "dep_fevrier", "dep_mars", "dep_avril",
  "dep_mai", "dep_juin", "dep_juillet", "dep_aout",
  "dep_sept", "dep_oct", "dep_nov", "dep_dec"
]

# Calcul les dépenses totales par mois (somme sur la population)
total_mensuel = all_expenses_clean[var_mensuelles].sum()

# Création du graphique
plt.figure(figsize=(12, 6))
plt.bar(total_mensuel.index,total_mensuel.values)

# Titre et légendes
plt.title("Répartition des dépenses de santé par mois (niveau population)", fontsize=14)
plt.xlabel("Mois", fontsize=12)
plt.ylabel("Dépenses totales (USD)", fontsize=12)

# Rotation des labels pour plus de lisibilité
plt.xticks(rotation=45, ha='right')

# Affichage
plt.tight_layout()
plt.show()






Nous créons ensuite un graphique détaillant la proportion de valeurs différentes de NA ou 0 pour chaque poste de dépense.
Nous remarquons tout d'abord qu'un nombre important d'individus présente des dépenses nulles pour certains postes, voire pour le total des dépenses.
Il apparait également que les dépenses en pharmacie, de consultations et dentaires sont les plus fréquentes et les dépenses d'hospitalisation ou de soins à domicile sont les moins fréquentes.

In [ ]:


# Liste des postes à analyser
depenses_postes = [
    "exp_dental_total",
    "exp_hospitals_2022_total",
    "exp_outpatient_total",
    "exp_er_total",
    "exp_medicines_total",
    "exp_office_total",
    "exp_home_total",
    "exp_others_total", "TOTEXP22", "TOTEXP23"
]

# Proportion de valeurs non-NA et différentes de zéro
non_na_or_zero = (
    all_expenses_clean[depenses_postes].notna() &
    (all_expenses_clean[depenses_postes] != 0)
).mean()

# Proportion de valeurs NA (strictement NA)
na = 1 - non_na_or_zero

# Création du graphique
plt.figure(figsize=(14, 6))

plt.bar(non_na_or_zero.index, non_na_or_zero.values, label="Non-NA ou 0")
plt.bar(na.index, na.values, bottom=non_na_or_zero.values, label="NA ou 0")

plt.title("Proportion de NA vs Non-NA or Zero par poste de dépense", fontsize=14)
plt.ylabel("Proportion", fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend()

plt.tight_layout()
plt.show()



Enfin, nous traçons la courbe cumulée des dépenses de santé individuelles en 2022. Nous remarquons que la répartition est très inégale avec un faible pourcentage d'individus portant la très grande majorité des dépenses de santé. Par exemple:
- 1% des individus dépensent 20% de la somme totale des dépenses de santé en 2022
- 3% des individus dépensent 40% de la somme totale des dépenses de santé en 2022

In [ ]:
# Tri croissant des dépenses de santé en 2022
sorted_cost = np.sort(all_expenses_clean["TOTEXP22"])

# Part cumulée des dépenses individuelles
cum_individuals = np.arange(1, len(sorted_cost)+1) / len(sorted_cost)
cum_cost = np.cumsum(sorted_cost) / sorted_cost.sum()

plt.figure(figsize=(10,7))

# Tracé des axes
plt.xlim(0, 1)
plt.ylim(0, 1)

plt.axhline(0, color='black', linewidth=1)
plt.axvline(0, color='black', linewidth=1)

# Courbe de Lorenz
plt.plot(cum_individuals, cum_cost)
plt.plot([0,1],[0,1],'--', color="red")

# Seuils de dépenses
y_seuils = [0.2, 0.4, 0.6, 0.8]
x_seuils = [cum_individuals[np.searchsorted(cum_cost, y)] for y in y_seuils]

for x, y in zip(x_seuils, y_seuils):

    # Lignes verticales/horizontales
    plt.vlines(x, ymin=0, ymax=y, color='black', linestyle='--')
    plt.hlines(y, xmin=0, xmax=x, color='black', linestyle='--')

    # Point d'intersection
    plt.scatter(x, y, color='black', zorder=5)

    # Coordonnées des points d'intersection avec les axes dans des bulles
    plt.annotate(
        f"({x:.2f}, {y:.2f})",
        xy=(x, y),                          # point visé
        xytext=(x + 0.05, y + 0.05),        # position du label
        textcoords="data",
        fontsize=11,
        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black"),
        arrowprops=dict(
            arrowstyle="->",                # flèche simple
            color="black",
            lw=1.2
        )
    )

plt.title("Courbe de Lorenz des dépenses de santé individuelles en 2022", fontsize=15)
plt.xlabel("Part d'individus")
plt.ylabel("Part cumulée des dépenses")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


# 3.Prédiction du profil des individus en termes de dépenses de santé
Nous nous proposons à présent de prédire le profil des individus au regard de leur dépenses de santé en 2023.
Pour ce faire, nous allons créer une variable catégorielle attribuant un profil à chaque individu (profil que nous chercherons à prévoir).

*   Profil 1 : individu à coût très élevé (appartenant aux individus ayant les dépenses de santé parmi les 40% les plus élevées en 2023)
*   Profil 2 : individu à coût modéré (appartenant aux deuxième et troisième quintiles de coûts de santé en 2023)
*   Profil 3 : individu à coût faible (appartenant aux premier quintile de coût de santé en 2023)





In [ ]:
# Nous créons une variable Profil donnant le profil de chaque individu au regard
# de ses dépenses de santé en 2023
# Cette variable constitura notre variable cible

#On copie le dataframe pour éviter d'écraser ou mélanger les index (identifiants des lignes)
df = all_expenses_clean.copy()

#Nous trions les individus par coûts (dépenses de santé) en 2023
df_ordonne = df.sort_values(by="TOTEXP23").reset_index(drop=True)

# Calcul de la part cumulée des dépenses
df_ordonne["cum_cost_share"] = df_ordonne["TOTEXP23"].cumsum() / df_ordonne["TOTEXP23"].sum()

# Seuils
seuils = [0.2, 0.4, 0.6, 0.8, 1.0]

# Fonction pour définir le profil
def assign_bucket(cum_cost):
    if cum_cost <= seuils[0]: # individu à coût faible
        return 0
    elif cum_cost <= seuils[1]: # individu à coût modéré
        return 1
    else:                     #individu à coût élevé
        return 2

#On applique la fonction assign_bucket à la colonne cum_cost_share de df_ordonne
#pour créer la variable profil
df_ordonne["profil"] = df_ordonne["cum_cost_share"].apply(assign_bucket)

# Grâce à la commande .sort_index(), on revient à l'ordre donné par les index
# pour que l'assignation de la nouvelle se fasse correctement
all_expenses_clean["profil"] = df_ordonne.sort_index()["profil"]



In [ ]:
# Affichage du nombre d'individus de chaque profil
all_expenses_clean["profil"].value_counts().sort_index()

In [ ]:
# Sauvegarde
all_expenses_clean.to_csv("all_expenses_profil.csv", index=False)

In [ ]:
#Préparation des données pour les algorithmes de machine learning

##############################################################################
####  Définition de la variable cible et des attributs              #########
############################################################################

df = all_expenses_clean.copy()

# Variable cible : profil
y = df["profil"]

# Variables explicatives
X = df.drop([
    "TOTEXP23",
    "profil", "DUPERSID"
], axis=1, errors="ignore")



###############################################################################
#####    Création des échantillons test et train                        #######
###############################################################################

#On sépare les données en deux échantillons train et test

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    stratify=y,    # en cas de classification, cette option sert à conserver les mêmes prportions de chaque classe dans les échantillons d'entrainement et de test
    random_state=42 #utile pour la reproductibilité
)

n_samples, n_features = X_train.shape
print("L'échnatillon d'entrainement contient: {} individus et {} variables explicatives".format(n_samples, n_features))
print("L'échantillon test contient : {} individus".format(X_test.shape[0]))

In [ ]:
#Nous créons un transformeur preprocess pour prétraiter les données

###############################################################################
############              Preprocessing                               #########
###############################################################################

# Séparation des variables numériques et catégorielles pour le pipeline
num_vars = X.select_dtypes(include=["int64", "float64"]).columns
cat_vars = X.select_dtypes(include=["object", "category"]).columns

###############################################################################
############   Preprocess classique sans log-transformation           #########
###############################################################################

# Les variables numériques correspondent à des variables de santé:
# nous choisissons donc d'attribuer la valeur 0 à leur valeurs manquantes
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    #on utilise la transformation  (x-median)/ intervalle interquartile pour
    #tenir compte de la distribution très asymétrique des données de sante
    ("scaler", RobustScaler())
])

# Il n'y a aucune valeur manquante parmi les variables catégorielles choisies
# ce qui rend l'imputation facultative
categorical_transformer = Pipeline(steps=[
    #("imputer", SimpleImputer(strategy="constant", fill_value="Not defined")),
#encodage des variables catégorielles sous forme d'indicatrices
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

#On assemble le prétatraitement des variables numériques et cétegorielles
#dans un preprocesseur unique
preproc = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_vars),
        ("cat", categorical_transformer, cat_vars)
    ]
)

###############################################################################
############   Preprocess2 avec  log-transformation                   #########
###############################################################################

# Nous séparons l'unique variable négative(tendance), des autres variables
# toutes positives avant d'appliquer la log-transformation
num_negative = ["tendance"]
num_positive = [col for col in num_vars if col not in num_negative]

log_transformer = FunctionTransformer(lambda x: np.log1p(x))

numeric_positive_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("log", log_transformer),
    ("scaler", StandardScaler())
])

numeric_negative_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preproc2 =  ColumnTransformer(
    transformers=[
        ("num_pos", numeric_positive_transformer, num_positive),
        ("num_neg", numeric_negative_transformer, num_negative),
        ("cat", categorical_transformer, cat_vars)
    ],
    remainder="drop"
)


###############################################################################
############   Preprocess3 pour les réseaux de neurones               #########
###############################################################################

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("scaler", StandardScaler())
])

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocess3 = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_vars),
        ("cat", categorical_transformer, cat_vars)
    ]
)


## Premier modèle: régression logistique

In [ ]:
# Pipeline : Prétraitement + régression logistique

cols_to_remove = ["DUPERSID"]

pipe_logreg = Pipeline([
    ('preprocess', preproc),   # Utilisation du transformer preprocess créé au début de la section 3
    ('logreg', LogisticRegression(max_iter=5000, random_state=42)) #Entrainement du modèle sur les données transfomées
])


# Grille d’hyperparamètres pour la régression logistique
parameters_logreg = {
    'logreg__C': np.logspace(-8, 8, 17, base=2)
}

# Grid Search
clf_logreg = GridSearchCV(
    estimator=pipe_logreg,
    param_grid=parameters_logreg,
    cv=5,
    scoring="balanced_accuracy",
    n_jobs=-1
)

# Entraînement
clf_logreg.fit(X_train, y_train)

# Résultats
print("=== Résultats régression logistique===")
print("Meilleur paramètre (C) :", clf_logreg.best_params_)
print("Accuracy moyenne CV :", clf_logreg.best_score_) #performance lors de la validation croisée
print("Accuracy sur test :", clf_logreg.score(X_test, y_test)) #Vraie performance du modèle sur des données qui n'ont jamais été vues ni lors de l'entrainement, ni lors de la validation croisée


In [ ]:
#Evaluation des performances du modèle

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    classification_report,
    f1_score,
    top_k_accuracy_score
)

# Prédictions
y_pred = clf_logreg.predict(X_test)

print("\n=== Performances globales du modèle ===")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Balanced accuracy :", balanced_accuracy_score(y_test, y_pred))
print("Macro F1-score :", f1_score(y_test, y_pred, average="macro"))

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

print("\n=== Confusion Matrix ===")

# Matrice de confusion brute
cm = confusion_matrix(y_test, y_pred)

# Matrice de confusion formatée avec labels explicites
cm_df = pd.DataFrame(
    cm,
    index=[f"Vrai {c}" for c in sorted(y_test.unique())],
    columns=[f"Prédit {c}" for c in sorted(y_test.unique())]
)

print(cm_df)


## 3.2 Deuxième modèle de classification: le modèle SVM

In [ ]:
# Pipeline : Prétraitement + SVM
pipe_svm = Pipeline([
    ('preprocess', preproc2),   # Utilisation du transformer preprocess créé au début de la section 3
    ('svc', LinearSVC(max_iter=5000, dual=False, random_state=42)) #Entrainement du modèle sur les données transfomées
])

# Grille d’hyperparamètres pour SVM
parameters_svm = {'svc__C': [0.01, 0.1, 1, 10, 100]}

# Grid Search
clf_svm = GridSearchCV(
    estimator=pipe_svm,
    param_grid=parameters_svm,
    cv=5,
  #  scoring="balanced_accuracy",
    n_jobs=-1 #utilise tous les CPU disponibles
)

# Entraînement
clf_svm.fit(X_train, y_train)

# Résultats
print("=== Résultats SVM ===")
print("Meilleur paramètre (C) :", clf_svm.best_params_)
print("Accuracy moyenne CV :", clf_svm.best_score_) #performance lors de la validation croisée
print("Accuracy sur test :", clf_svm.score(X_test, y_test)) #Vraie performance du modèle sur des données qui n'ont jamais été vues ni lors de l'entrainement, ni lors de la validation croisée


In [ ]:
#Evaluation des performances du modèle

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    classification_report,
    f1_score,
    top_k_accuracy_score
)

# Prédictions
y_pred = clf_svm.predict(X_test)

print("\n=== Performances globales du modèle ===")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Balanced accuracy :", balanced_accuracy_score(y_test, y_pred))
print("Macro F1-score :", f1_score(y_test, y_pred, average="macro"))

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

print("\n=== Confusion Matrix ===")

# Matrice de confusion brute
cm = confusion_matrix(y_test, y_pred)

# Matrice de confusion formatée avec labels explicites
cm_df = pd.DataFrame(
    cm,
    index=[f"Vrai {c}" for c in sorted(y_test.unique())],
    columns=[f"Prédit {c}" for c in sorted(y_test.unique())]
)

print(cm_df)



## 3.3 troisième modèle: Random Forest

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Pipeline : prétraitement + Random Forest
pipe_rf = Pipeline([
    ('preprocess', preproc),
    ('rf', RandomForestClassifier(
        random_state=42,
        n_jobs=-1
    ))
])

# Grille d'hyperparamètres pour Random Forest
parameters_rf = {
    'rf__n_estimators': [200, 500],          # nombre d'arbres
    'rf__max_depth': [None, 10, 20],         # profondeur maximale
    'rf__min_samples_split': [2, 5, 10],     # min d'échantillons pour un split
    'rf__min_samples_leaf': [1, 2, 4],       # min d'échantillons par feuille
    'rf__max_features': ['sqrt', 'log2', 0.5]  # nombre de features testés par split
}

# Grid Search : recherche des hyperparamètres par validation croisée
clf_rf = GridSearchCV(
    estimator=pipe_rf,
    param_grid=parameters_rf,
    cv=5,
    scoring="balanced_accuracy",
    n_jobs=-1,
    error_score='raise'
)

# Entraînement
clf_rf.fit(X_train, y_train)

# Résultats
print("=== Résultats Random Forest ===")
print("Meilleurs paramètres :", clf_rf.best_params_)
print("Balanced Accuracy CV :", clf_rf.best_score_)
print("Accuracy test :", clf_rf.score(X_test, y_test))


## Modèle 4 : XGBoost

In [ ]:
import warnings
warnings.filterwarnings("ignore")


# Encodage des labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

# Vérification des classes
print("Classes encodées :", np.unique(y_train_enc))

# Pipeline
pipe_xgb = Pipeline([
    ('preprocess', preproc),
    ('xgb', XGBClassifier(
        objective='multi:softprob',
        num_class=3,
        eval_metric='mlogloss',
        random_state=42,
        tree_method="hist",
        n_jobs=-1
    ))
])

# Grille
parameters_xgb = {
    'xgb__n_estimators': [200, 500],
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.01, 0.1, 0.3],
    'xgb__subsample': [0.6, 0.8, 1.0],
    'xgb__colsample_bytree': [0.6, 0.8, 1.0]
}

# Grid Search
clf_xgb = GridSearchCV(
    estimator=pipe_xgb,
    param_grid=parameters_xgb,
    cv=5,
    scoring="balanced_accuracy",
    n_jobs=-1,
    error_score="raise"
)

# Entraînement
clf_xgb.fit(X_train, y_train_enc)

# Résultats
print("=== Résultats XGBoost ===")
print("Meilleurs paramètres :", clf_xgb.best_params_)
print("Balanced Accuracy CV :", clf_xgb.best_score_)
print("Accuracy test :", clf_xgb.score(X_test, y_test_enc))


## 3.5 Réseau de neurones / Multilayer Perceptron (MLP)

In [ ]:
from tqdm.notebook import tqdm


# Création d'un module PyTorch qui représente un modèle.
# input_size : nombre de features en entrée
# hidden_sizes : tailles des couches cachées (2 couches : 128 puis 64 neurones)
# output_size : nombre de classes à prédire (par défaut 3)

###############################################################################
###   Préparation des données pour Pythorch                                ###
##############################################################################

class SimpleFeedForward(nn.Module):
    def __init__(self, input_size, hidden_sizes=[128, 64], output_size=3):
        super().__init__()
      #Définition de l'architecture du réseau
      # Linear → ReLU → Linear → ReLU → Linear
        self.classifier = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[1], output_size)
        )

    def forward(self, x):
        return self.classifier(x)

# Le modèle apprend les paramètres utiles aux transformations (médiane,
# écart_interquartile...) sur le train uniquement, ces données seront utilisées
# à l'étape suivante
preprocess3.fit(X_train)

# Transformation des données avec le preprocessing
X_train_t = preprocess3.transform(X_train)
X_test_t  = preprocess3.transform(X_test)

# Conversion en tenseurs
X_train_tensor = torch.tensor(X_train_t, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test_t, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.long)

# DataLoader (générateur de mini-batch)
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
test_ds  = TensorDataset(X_test_tensor, y_test_tensor)

trainloader = DataLoader(train_ds, batch_size=64, shuffle=True)
valloader  = DataLoader(test_ds, batch_size=64)

###############################################################################
###                      Boucle d'entrainement                              ###
##############################################################################

def train(model, trainloader, loss_fn, optimizer, epoch, num_epochs):
    model.train()
    loop = tqdm(trainloader, desc=f"Training Epoch [{epoch+1}/{num_epochs}]")

    for inputs, targets in loop:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()


# ============================================================
# 3. Initialisation du modèle, de la loss et de l’optimiseur
# ============================================================

input_size = X_train_t.shape[1]   # Nombre de colonnes après preprocessing
output_size = 3                   # Classes du profil (0/1/2)
hidden_sizes = [128, 64]
lr = 1e-3
num_epochs = 20

net = SimpleFeedForward(input_size, hidden_sizes, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)



###############################################################################
###                     Validation du modèle                               ###
##############################################################################

def validation(model, valloader, loss_fn):
    model.eval()
    total = 0
    running_loss = 0.0
    accuracy = 0.0

    with torch.no_grad():
        loop = tqdm(valloader, desc="Validation")

        for inputs, targets in loop:
            outputs = model(inputs)

            batch_size = inputs.shape[0]
            total += batch_size

            running_loss += batch_size * loss_fn(outputs, targets).item()
            accuracy += (outputs.argmax(dim=1) == targets).sum().item()

            loop.set_postfix(
                val_loss=running_loss / total,
                val_acc=accuracy / total
            )

    return running_loss / total, accuracy / total

###############################################################################
###                     Entrainement du modèle                              ###
##############################################################################

input_size = X_train_t.shape[1]   # nb de colonnes après preprocess
model = SimpleFeedForward(input_size=input_size, hidden_sizes=[128, 64], output_size=3)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 20

for epoch in range(num_epochs):
    train(model, trainloader, criterion, optimizer, epoch, num_epochs)
    val_loss, val_acc = validation(model, valloader, criterion)

# ============================================================
# 5. Boucle d'entraînement sur le dataset de santé
# ============================================================

for epoch in range(num_epochs):

    train(net, trainloader, criterion, optimizer, epoch, num_epochs)

    val_loss, val_acc = validation(net, valloader, criterion)

    print(f"Epoch {epoch+1}/{num_epochs} - Validation loss: {val_loss:.4f} - acc: {val_acc:.4f}")

# ============================================================
# 6. Évaluation finale sur le jeu de test (jamais vu)
# ============================================================

test_loss, test_acc = validation(net, valloader, criterion)
print(f"Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")




In [ ]:
X_train_t.max(), X_train_t.min(), np.isnan(X_train_t).sum()


In [ ]:
# Création des classes d'âge
bins = [0, 15, 25, 35, 45,55, 65,75,85,  120]  # limites
labels = ["0–15","15-25","25-35", "35–45", "45–55","55-65","65-75", "75-85", "85+"]  # noms des classes

all_expenses_clean["age_class"] = pd.cut(
    all_expenses_clean["AGE22X"],
    bins=bins,
    labels=labels,
    right=False  # intervalle fermé à gauche [ )
)

dep_age = all_expenses_clean.groupby("age_class")["TOTEXP22"].sum()

plt.figure(figsize=(10,5))
plt.bar(dep_age.index, dep_age.values)

plt.title("Répartition des dépenses totales de santé par tranches d'âge en 2022", fontsize=14)
plt.xlabel("Âge")
plt.ylabel("Dépenses totales (USD)")
plt.grid(axis='y')
plt.tight_layout()
plt.show()